In [8]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import ConfigurableField
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_ollama.llms import OllamaLLM
from typing import Tuple, List, Optional
from langchain.chains import GraphCypherQAChain
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain import hub
from langchain.schema.runnable import RunnablePassthrough
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.prompts import SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.prompts import ChatMessagePromptTemplate, PromptTemplate

from neo4j import GraphDatabase
from langchain_community.vectorstores import Neo4jVector
from langchain_community.graphs import Neo4jGraph
from langchain.vectorstores import FAISS

import pickle
import os

In [3]:
#loading the embedding model from huggingface
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name,
  model_kwargs=model_kwargs
)

C:\Users\chand\AppData\Local\Temp\ipykernel_15768\269612671.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
C:\Users\chand\.conda\envs\gastro\Lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [4]:
# Load from local storage
persisted_vectorstore = FAISS.load_local("faiss_index_", embeddings,allow_dangerous_deserialization=True)

In [5]:
#creating a retriever on top of database
retriever = persisted_vectorstore.as_retriever()

In [6]:
fusion_template = """
        Task: You are an assistant that generates multiple variations of a given question. 
        For each variation, maintain the original intent of the question, but change the phrasing, structure, 
        or tone to create a diverse set of queries.

Generate 5-7 variations that cover:

Synonym replacements while keeping the question concise.
Alternative structures, such as rephrasing into "why," "how," or "what" forms if relevant.
Casual and formal tones.
Slightly more specific or broader wording.
Examples:

Original Question: "What is the impact of inflation on the stock market?"
Variations:
"How does inflation affect stock prices?"
"What are the effects of inflation on the stock market?"
"In what ways does inflation influence stock market trends?"
"Could inflation lead to changes in stock market values?"
"How does rising inflation impact the performance of stocks?"
"What influence does inflation have on market prices?"
"What happens to stock prices when inflation increases?"

the final answer should be a python list: [
   "How does inflation affect stock prices?",
    "What are the effects of inflation on the stock market?",
    "In what ways does inflation influence stock market trends?",
    "Could inflation lead to changes in stock market values?",
    "How does rising inflation impact the performance of stocks?",
    "What influence does inflation have on market prices?",
    "What happens to stock prices when inflation increases?"
]

Now generate the list of variations for the given question
        """

In [13]:
original_query = "What are the new AI regulations in Europe"

In [14]:
prompt = ChatPromptTemplate(input_variables=['original_query'],
                            messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[],template=fusion_template)),
                            HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['original_query'], template='Generate multiple search queries related to: {question} \n OUTPUT (7 queries):'))])

In [15]:
model = OllamaLLM(model="llama3.2")

In [16]:
generate_queries = (
    prompt | model | StrOutputParser() | (lambda x: x.split("\n"))
)

In [17]:
print(generate_queries)

first=ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='\n        Task: You are an assistant that generates multiple variations of a given question. \n        For each variation, maintain the original intent of the question, but change the phrasing, structure, \n        or tone to create a diverse set of queries.\n\nGenerate 5-7 variations that cover:\n\nSynonym replacements while keeping the question concise.\nAlternative structures, such as rephrasing into "why," "how," or "what" forms if relevant.\nCasual and formal tones.\nSlightly more specific or broader wording.\nExamples:\n\nOriginal Question: "What is the impact of inflation on the stock market?"\nVariations:\n"How does inflation affect stock prices?"\n"What are the effects of inflation on the stock market?"\n"In what ways does inflation influence stock market tren

In [18]:
from langchain.load import dumps, loads


def reciprocal_rank_fusion(results: list[list], k=60):
    fused_scores = {}
    for docs in results:
        # Assumes the docs are returned in sorted order of relevance
        for rank, doc in enumerate(docs):
            doc_str = dumps(doc)
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            previous_score = fused_scores[doc_str]
            fused_scores[doc_str] += 1 / (rank + k)

    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]
    return reranked_results

In [19]:
ragfusion_chain = generate_queries | retriever.map() | reciprocal_rank_fusion

In [20]:
import langchain
langchain.debug = True

In [21]:
ragfusion_chain.input_schema.schema()

{'properties': {'question': {'title': 'Question', 'type': 'string'}},
 'required': ['question'],
 'title': 'PromptInput',
 'type': 'object'}

In [23]:
ragfusion_chain.invoke({"question": original_query})

[chain/start] [chain:RunnableSequence] Entering Chain run with input:
{
  "question": "What are the new AI regulations in Europe"
}
[chain/start] [chain:RunnableSequence > prompt:ChatPromptTemplate] Entering Prompt run with input:
{
  "question": "What are the new AI regulations in Europe"
}
[chain/end] [chain:RunnableSequence > prompt:ChatPromptTemplate] s] Exiting Prompt run with output:
[outputs]
[llm/start] [chain:RunnableSequence > llm:OllamaLLM] Entering LLM run with input:
{
  "prompts": [
    "System: \n        Task: You are an assistant that generates multiple variations of a given question. \n        For each variation, maintain the original intent of the question, but change the phrasing, structure, \n        or tone to create a diverse set of queries.\n\nGenerate 5-7 variations that cover:\n\nSynonym replacements while keeping the question concise.\nAlternative structures, such as rephrasing into \"why,\" \"how,\" or \"what\" forms if relevant.\nCasual and formal tones.\nSl

ConnectError: [WinError 10061] No connection could be made because the target machine actively refused it